In [1]:
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random

database = "/home/wmnlab/D/database/"
date = "2022-12-26"
devices = sorted([
    # "sm00",
    # "sm01",
    # "sm02",
    # "sm03",
    # "sm04",
    # "sm05",
    # "sm06",
    "sm07",
    "sm08",
    "qc00",
    "qc01",
    # "qc02",
    # "qc03",
])
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "tsync": (1, None),
    # "_Bandlock_Udp": (4, ["#01", "#02", "#03", "#04"]),
    # "_Bandlock_Udp": (4, ["#03", "#04", "#05", "#06"]),
    # "_Bandlock_Udp": (4, []),
    # "_Bandlock_Udp": (6, []),
    # "_Bandlock_Udp_B1_B3":  (1, ["#01"]),
    # "_Bandlock_Udp_B1_B3":  (6, []),
    # "_Bandlock_Udp_B3_B28": (2, []),
    # "_Bandlock_Udp_B28_B1": (2, []),
    # "_Bandlock_Udp_B1_B3": (4, []),
    # "_Bandlock_Udp_B3_B7": (4, []),
    # "_Bandlock_Udp_B7_B8": (4, []),
    # "_Bandlock_Udp_B8_B1": (4, []),
    "_Modem_Phone_Comparative_Exeriments": (6, []),
}

class Payload:
    LENGTH = 250              # (Bytes)
    TAG = "000425d401df5e76"  # 2 71828 3 1415926 (hex)            : 8-bytes
    OFS_TIME = (16, 24)       # epoch time of 'yyyy/mm/dd hh:mm:ss': 4-bytes
    OFS_USEC = (24, 32)       # microsecond (usec)                 : 4-bytes
    OFS_SEQN = (32, 40)       # sequence number (start from 1)     : 4-bytes
class ServerIP:
    PUBLIC = "140.112.20.183"  # 2F    
    PRIVATE = "192.168.1.251"  # 2F
    # PRIVATE = "192.168.1.248"  # 2F previous
    # PUBLIC = "140.112.17.209"  # 3F
    # PRIVATE = "192.168.1.108"  # 3F

DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / Payload.LENGTH / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")
INF = 2147483647

packet_rate (pps): 500.0 



In [2]:
def to_utc8(ts):
    """
    Convert an epoch time into a readable format.
    Switch from utc-0 into utc-8.
    
    Args:
        ts (float): timestamp composed of datetimedec + microsecond (e.g., 1644051509.989306)
    Returns:
        (datetime.datetime): a readable timestamp (utc-8)
    """
    return (dt.datetime.utcfromtimestamp(ts) + dt.timedelta(hours=8))

def str_to_datetime(ts):
    """
    Convert a timestamp string in microseconds or milliseconds into datetime.datetime

    Args:
        ts (str): timestamp string (e.g., 2022-09-29 16:24:58.252615)
    Returns:
        (datetime.datetime)
    """
    try:
        ts_datetime = dt.datetime.strptime(ts, '%Y-%m-%d %H:%M:%S.%f')
    except:
        ts_datetime = dt.datetime.strptime(ts, '%Y-%m-%d %H:%M:%S')
    return ts_datetime

def datetime_to_str(ts):
    """
    Convert a datetime timestamp in microseconds into str

    Args:
        ts (datetime.datetime): datetime timestamp (e.g., datetime.datetime(2022, 9, 29, 16, 24, 58, 252615))
    Returns:
        (str): timestamp string (e.g., 2022-09-29 16:24:58.252615)
    """
    try:
        ts_string = dt.datetime.strftime(ts, '%Y-%m-%d %H:%M:%S.%f')
    except:
        ts_string = dt.datetime.strftime(ts, '%Y-%m-%d %H:%M:%S')
    return ts_string

In [3]:
dl_txdf = pd.read_csv("/home/wmnlab/D/database/2023-02-04#2/_Bandlock_Udp_all_RM500Q/qc01/#01/middle/udp_dnlk_server_pkt_brief.csv")
dl_rxdf = pd.read_csv("/home/wmnlab/D/database/2023-02-04#2/_Bandlock_Udp_all_RM500Q/qc01/#01/middle/udp_dnlk_client_pkt_brief.csv")
ul_txdf = pd.read_csv("/home/wmnlab/D/database/2023-02-04#2/_Bandlock_Udp_all_RM500Q/qc01/#01/middle/udp_uplk_client_pkt_brief.csv")
ul_rxdf = pd.read_csv("/home/wmnlab/D/database/2023-02-04#2/_Bandlock_Udp_all_RM500Q/qc01/#01/middle/udp_uplk_server_pkt_brief.csv")

In [4]:
dl_txseq = list(dl_txdf["sequence.number"].array)
dl_rxseq = list(dl_rxdf["sequence.number"].array)
dlst = max(dl_txseq[0], dl_rxseq[0])
dlet = min(dl_txseq[-1], dl_rxseq[-1])
print(dlst, dlet)

ul_txseq = list(ul_txdf["sequence.number"].array)
ul_rxseq = list(ul_rxdf["sequence.number"].array)
ulst = max(ul_txseq[0], ul_rxseq[0])
ulet = min(ul_txseq[-1], ul_rxseq[-1])
print(ulst, ulet)

st = max(dlst, ulst)
et = min(dlet, ulet)
print("----------------")
st += PKT_RATE * 5  # 開頭切5秒
et -= PKT_RATE * 5  # 結尾切5秒
print(st, et)

1006 279172
1 279164
----------------
3506.0 276664.0


In [5]:
dl_txdf = dl_txdf[(dl_txdf["sequence.number"] >= st) & (dl_txdf["sequence.number"] <= et)]
dl_rxdf = dl_rxdf[(dl_rxdf["sequence.number"] >= st) & (dl_rxdf["sequence.number"] <= et)]
ul_txdf = ul_txdf[(ul_txdf["sequence.number"] >= st) & (ul_txdf["sequence.number"] <= et)]
ul_rxdf = ul_rxdf[(ul_rxdf["sequence.number"] >= st) & (ul_rxdf["sequence.number"] <= et)]

dl_txdf.reset_index(drop=True, inplace=True)
dl_rxdf.reset_index(drop=True, inplace=True)
ul_txdf.reset_index(drop=True, inplace=True)
ul_rxdf.reset_index(drop=True, inplace=True)

In [6]:
def get_loss(txdf, rxdf):
    rxdf['Timestamp'] = pd.to_datetime(rxdf['Timestamp'])  # arrival.time
    rxdf['payload.time'] = pd.to_datetime(rxdf['payload.time'])  # payload.time

    timestamp_list = list(map(list, zip(rxdf['sequence.number'].astype(int).array, rxdf['Timestamp_epoch'].astype(float).array)))
    timestamp_store = None
    loss_timestamp_list = []
    count = 0  # to count the total number of packet losses
    # _eseq = 1  # next expected sequence number
    _eseq = timestamp_list[0][0] # next expected sequence number
    for i in tqdm(range(len(rxdf))):
        timestamp = timestamp_list[i]
        if timestamp[0] == _eseq:
            ### received packet's sequence number as expected
            pass
        else:
            ### packet losses occur
            ### 可處理連續掉 N 個封包的狀況
            ### timestamp_store: 前一刻收到的封包
            ### timestamp: 此時此刻收到的封包
            ### _eseq 為預期收到的封包 sequence number (前一刻收到的 seq number + 1)
            ### rxdf.loc[i, 'sequence.number'] 為此時此刻收到的封包 seq
            ### rxdf.loc[i, 'sequence.number']-pointer+2 == 遺漏的封包數+2 (頭+尾)，因此要去頭去尾才是實際遺漏的封包
            n = timestamp[0] - _eseq + 2
            # if timestamp_store == None:
            #     ### if the first-N packets lost, we cannot predict the loss timestamp, so we only record their sequemce number.
            #     loss_linspace = np.linspace([0, timestamp[1] - (n-1) / PKT_RATE], timestamp, n)
            # else:
            #     loss_linspace = np.linspace(timestamp_store, timestamp, n)
            loss_linspace = np.linspace(timestamp_store, timestamp, n)
            loss_linspace = loss_linspace[1:-1]  # 去頭去尾
            for item in loss_linspace:
                count += 1
                loss_time = [round(item[0]), to_utc8(item[1]), item[1]]  # (expected) arrival timestamp
                loss_timestamp_list.append(loss_time)
        # Update information
        timestamp_store = timestamp
        _eseq = timestamp[0] + 1
    
    ### add payload, transmit timestamp
    tx_ts_arr = list(zip(txdf['sequence.number'].array, txdf['payload.time'].array, txdf['payload.time_epoch'].array, txdf['Timestamp'].array, txdf['Timestamp_epoch'].array))
    j = 0
    N = len(loss_timestamp_list)
    for i in tqdm(range(len(tx_ts_arr))):
        if N == 0:
            break
        if tx_ts_arr[i][0] == loss_timestamp_list[j][0]:
            loss_timestamp_list[j].append(tx_ts_arr[i][1])
            loss_timestamp_list[j].append(tx_ts_arr[i][2])
            loss_timestamp_list[j].append(tx_ts_arr[i][3])
            loss_timestamp_list[j].append(tx_ts_arr[i][4])
            j += 1
            if j == N:
                break
    ### 因為是由 Client 端主動開始和結束實驗，且程式邏輯為: 開 Tcpdump -> 開 iperf -> 關 Tcpdump -> 關 iperf
    ### 因此 Uplink TX 的 MAX_SEQ 會比 RX 小，Downlink TX 的 MAX_SEQ 會比 RX 大。
    loss_timestamp_list = [item for item in loss_timestamp_list if len(item) == 7]
    # pprint(loss_timestamp_list)
    
    # N = len(loss_timestamp_list)
    loss_timestamps = list(map(list, zip(*loss_timestamp_list)))
    df = pd.DataFrame.from_dict(
        {
            "sequence.number": loss_timestamps[0],
            "Timestamp": loss_timestamps[3],  # payload.time
            "Timestamp_epoch": loss_timestamps[4],  # payload.time_epoch
            "lost": [True] * N,
            "latency": [INF] * N,
            "transmit.time": loss_timestamps[5],
            "transmit.time_epoch": loss_timestamps[6],
            "arrival.time": loss_timestamps[1],
            "arrival.time_epoch": loss_timestamps[2],
        }
    )
    # print(df)
    return df

In [7]:
lossdf = get_loss(dl_txdf.copy(), dl_rxdf.copy())
display(lossdf)

 88%|████████▊ | 239605/273159 [00:00<00:00, 3708659.32it/s]


,sequence.number,Timestamp,Timestamp_epoch,lost,latency,transmit.time,transmit.time_epoch,arrival.time,arrival.time_epoch
0,90786,2023-02-04 16:54:24.091612,1.675501e+09,True,2147483647,2023-02-04 16:54:24.091621,1.675501e+09,2023-02-04 16:54:22.338074,1.675501e+09
1,90787,2023-02-04 16:54:24.093612,1.675501e+09,True,2147483647,2023-02-04 16:54:24.093622,1.675501e+09,2023-02-04 16:54:22.340091,1.675501e+09
2,90788,2023-02-04 16:54:24.095615,1.675501e+09,True,2147483647,2023-02-04 16:54:24.095625,1.675501e+09,2023-02-04 16:54:22.342108,1.675501e+09
3,90789,2023-02-04 16:54:24.097613,1.675501e+09,True,2147483647,2023-02-04 16:54:24.097621,1.675501e+09,2023-02-04 16:54:22.344125,1.675501e+09
4,90790,2023-02-04 16:54:24.099613,1.675501e+09,True,2147483647,2023-02-04 16:54:24.099622,1.675501e+09,2023-02-04 16:54:22.346142,1.675501e+09
...,...,...,...,...,...,...,...,...,...
1024,243107,2023-02-04 16:59:28.747848,1.675501e+09,True,2147483647,2023-02-04 16:59:28.747861,1.675501e+09,2023-02-04 16:59:26.997083,1.675501e+09
1025,243108,2023-02-04 16:59:28.749848,1.675501e+09,True,2147483647,2023-02-04 16:59:28.749856,1.675501e+09,2023-02-04 16:59:26.999044,1.675501e+09
1026,243109,2023-02-04 16:59:28.751848,1.675501e+09,True,2147483647,2023-02-04 16:59:28.751857,1.675501e+09,2023-02-04 16:59:27.001005,1.675501e+09
1027,243110,2023-02-04 16:59:28.753849,1.675501e+09,True,2147483647,2023-02-04 16:59:28.753856,1.675501e+09,2023-02-04 16:59:27.002965,1.675501e+09


In [8]:
def get_match(txdf, rxdf):
    ### add transmitted timestamp
    
    # # print(len(txdf))
    # # print(len(rxdf))
    # j = 0
    # N = len(txdf)
    # M = len(rxdf)
    # rxdf = rxdf.reindex(rxdf.columns.tolist() + ['transmit.time', 'transmit.time_epoch'], axis=1)
    # for i in tqdm(range(len(rxdf))):
    #     while j != N and txdf.at[j, 'sequence.number'] != rxdf.at[i, 'sequence.number']:
    #         j += 1
    #     if j != N:
    #         rxdf.at[i, 'transmit.time'] = txdf.at[j, 'Timestamp']
    #         rxdf.at[i, 'transmit.time_epoch'] = txdf.at[j, 'Timestamp_epoch']
    # df = rxdf
    
    rxseq = rxdf['sequence.number'].array
    txseq = txdf['sequence.number'].array
    txts = txdf['Timestamp'].array
    txts_epoch= txdf['Timestamp_epoch'].array
    rx_txts_arr = []
    rx_txts_epoch_arr = []
    # print(len(txdf))
    # print(len(rxdf))
    j = 0
    N = len(txdf)
    M = len(rxdf)
    for i in tqdm(range(len(rxseq))):
        while j != N and txseq[j] != rxseq[i]:
            j += 1
        if j != N:
            rx_txts_arr.append(txts[j])
            rx_txts_epoch_arr.append(txts_epoch[j])
    df = rxdf.join(pd.DataFrame({'transmit.time' : rx_txts_arr, 'transmit.time_epoch' : rx_txts_epoch_arr}))
    
    df.dropna(how='any', subset=['transmit.time', 'transmit.time_epoch'], axis=0, inplace=True)
    df.rename(
        columns={
            "Timestamp": "arrival.time",
            "Timestamp_epoch": "arrival.time_epoch",
            "payload.time": "Timestamp",
            "payload.time_epoch": "Timestamp_epoch",
        }, inplace=True
    )
    df["lost"] = False
    df["latency"] = 0
    df = df[["sequence.number", "Timestamp", "Timestamp_epoch", "lost", "latency", "transmit.time", "transmit.time_epoch", "arrival.time", "arrival.time_epoch"]]
    
    return df

In [9]:
latndf = get_match(dl_txdf.copy(), dl_rxdf.copy())
display(latndf)

100%|██████████| 272130/272130 [00:00<00:00, 526507.17it/s]


,sequence.number,Timestamp,Timestamp_epoch,lost,latency,transmit.time,transmit.time_epoch,arrival.time,arrival.time_epoch
0,3506,2023-02-04 16:51:29.523455,1.675501e+09,False,0,2023-02-04 16:51:29.523463,1.675501e+09,2023-02-04 16:51:27.771660,1.675501e+09
1,3507,2023-02-04 16:51:29.525455,1.675501e+09,False,0,2023-02-04 16:51:29.525462,1.675501e+09,2023-02-04 16:51:27.771710,1.675501e+09
2,3508,2023-02-04 16:51:29.527455,1.675501e+09,False,0,2023-02-04 16:51:29.527463,1.675501e+09,2023-02-04 16:51:27.771711,1.675501e+09
3,3509,2023-02-04 16:51:29.529455,1.675501e+09,False,0,2023-02-04 16:51:29.529462,1.675501e+09,2023-02-04 16:51:27.777692,1.675501e+09
4,3510,2023-02-04 16:51:29.531455,1.675501e+09,False,0,2023-02-04 16:51:29.531463,1.675501e+09,2023-02-04 16:51:27.777694,1.675501e+09
...,...,...,...,...,...,...,...,...,...
272125,276660,2023-02-04 17:00:35.856984,1.675501e+09,False,0,2023-02-04 17:00:35.856991,1.675501e+09,2023-02-04 17:00:34.114199,1.675501e+09
272126,276661,2023-02-04 17:00:35.858984,1.675501e+09,False,0,2023-02-04 17:00:35.858991,1.675501e+09,2023-02-04 17:00:34.114253,1.675501e+09
272127,276662,2023-02-04 17:00:35.860984,1.675501e+09,False,0,2023-02-04 17:00:35.860992,1.675501e+09,2023-02-04 17:00:34.114254,1.675501e+09
272128,276663,2023-02-04 17:00:35.862984,1.675501e+09,False,0,2023-02-04 17:00:35.862991,1.675501e+09,2023-02-04 17:00:34.114255,1.675501e+09


In [10]:
df = pd.concat([lossdf, latndf], axis=0)
df.sort_values(by=["sequence.number"], inplace=True)
df.reset_index(drop=True, inplace=True)
display(df)

,sequence.number,Timestamp,Timestamp_epoch,lost,latency,transmit.time,transmit.time_epoch,arrival.time,arrival.time_epoch
0,3506,2023-02-04 16:51:29.523455,1.675501e+09,False,0,2023-02-04 16:51:29.523463,1.675501e+09,2023-02-04 16:51:27.771660,1.675501e+09
1,3507,2023-02-04 16:51:29.525455,1.675501e+09,False,0,2023-02-04 16:51:29.525462,1.675501e+09,2023-02-04 16:51:27.771710,1.675501e+09
2,3508,2023-02-04 16:51:29.527455,1.675501e+09,False,0,2023-02-04 16:51:29.527463,1.675501e+09,2023-02-04 16:51:27.771711,1.675501e+09
3,3509,2023-02-04 16:51:29.529455,1.675501e+09,False,0,2023-02-04 16:51:29.529462,1.675501e+09,2023-02-04 16:51:27.777692,1.675501e+09
4,3510,2023-02-04 16:51:29.531455,1.675501e+09,False,0,2023-02-04 16:51:29.531463,1.675501e+09,2023-02-04 16:51:27.777694,1.675501e+09
...,...,...,...,...,...,...,...,...,...
273154,276660,2023-02-04 17:00:35.856984,1.675501e+09,False,0,2023-02-04 17:00:35.856991,1.675501e+09,2023-02-04 17:00:34.114199,1.675501e+09
273155,276661,2023-02-04 17:00:35.858984,1.675501e+09,False,0,2023-02-04 17:00:35.858991,1.675501e+09,2023-02-04 17:00:34.114253,1.675501e+09
273156,276662,2023-02-04 17:00:35.860984,1.675501e+09,False,0,2023-02-04 17:00:35.860992,1.675501e+09,2023-02-04 17:00:34.114254,1.675501e+09
273157,276663,2023-02-04 17:00:35.862984,1.675501e+09,False,0,2023-02-04 17:00:35.862991,1.675501e+09,2023-02-04 17:00:34.114255,1.675501e+09


In [11]:
# try:
#     with open(os.path.join(database, date, "tsync", dev, "delta.txt"), encoding="utf-8") as f:
#         epoch_delta = float(f.readline())
# except:
#     epoch_delta = 0

In [12]:
with open("/home/wmnlab/D/database/2023-02-04#2/tsync/qc01/delta.txt", encoding="utf-8") as f:
    lines = f.readlines()
    timerec1 = pd.to_datetime(lines[0])
    epoch_delta1 = float(lines[1])
    timedelta1 = pd.Timedelta(seconds=epoch_delta1).round('us')
with open("/home/wmnlab/D/database/2023-02-04#2/tsync/qc01/delta1.txt", encoding="utf-8") as f:
    lines = f.readlines()
    timerec2 = pd.to_datetime(lines[0])
    epoch_delta2 = float(lines[1])
    timedelta2 = pd.Timedelta(seconds=epoch_delta2).round('us')

print(timerec1)
print(epoch_delta1)
print(timedelta1)
print(timerec2)
print(epoch_delta2)
print(timedelta2)
delta1 = (timerec1, epoch_delta1, timedelta1)
delta2 = (timerec2, epoch_delta2, timedelta2)

2023-02-04 14:14:45.965295
1.871482
0 days 00:00:01.871482
2023-02-04 17:38:02.814131
1.806751
0 days 00:00:01.806751


In [13]:
def interp(x, y, ratio):
    """
    Interpolation

    Args:
        x, y (datetime.datetime)
        ratio (float): a decimal numeral in a range [0, 1]
    Returns:
        (datetime.datetime): breakpoint of interpolation
    """
    return x + (y - x) * ratio

def get_compensate(df, mode, delta1=None, delta2=None):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # payload.time
    df['transmit.time'] = pd.to_datetime(df['transmit.time'])
    df['arrival.time'] = pd.to_datetime(df['arrival.time'])
    if mode == "dl":
        benchmark = list(df["transmit.time"].array)
    elif mode == "ul":
        benchmark = list(df["arrival.time"].array)
    bm1, bm2 = benchmark[0], benchmark[-1]
    ratio1 = (bm1-delta1[0]).total_seconds() / (delta2[0]-delta1[0]).total_seconds()
    ratio2 = (bm2-delta1[0]).total_seconds() / (delta2[0]-delta1[0]).total_seconds()
    _delta1 = interp(delta1[1], delta2[1], ratio1)
    _delta2 = interp(delta1[1], delta2[1], ratio2)
    epoch_comp_list = list(np.round(np.linspace(_delta1, _delta2, len(df)), 6))
    comp_list = pd.to_timedelta(epoch_comp_list, "sec")
    # print(comp_list)
    # display(df)
    if mode == "dl":
        df['arrival.time_epoch'] = df['arrival.time_epoch'].add(pd.Series(epoch_comp_list))
        df['arrival.time'] = df['arrival.time'].add(pd.Series(comp_list))
    elif mode == "ul":
        df['Timestamp_epoch'] = df['Timestamp_epoch'].add(pd.Series(epoch_comp_list))
        df['Timestamp'] = df['Timestamp'].add(pd.Series(comp_list))
        df['transmit.time_epoch'] = df['transmit.time_epoch'].add(pd.Series(epoch_comp_list))
        df['transmit.time'] = df['transmit.time'].add(pd.Series(comp_list))
    # display(df)
    return df

In [14]:
df = get_compensate(df.copy(), "dl", delta1, delta2)
display(df)

,sequence.number,Timestamp,Timestamp_epoch,lost,latency,transmit.time,transmit.time_epoch,arrival.time,arrival.time_epoch
0,3506,2023-02-04 16:51:29.523455,1.675501e+09,False,0,2023-02-04 16:51:29.523463,1.675501e+09,2023-02-04 16:51:29.593236,1.675501e+09
1,3507,2023-02-04 16:51:29.525455,1.675501e+09,False,0,2023-02-04 16:51:29.525462,1.675501e+09,2023-02-04 16:51:29.593286,1.675501e+09
2,3508,2023-02-04 16:51:29.527455,1.675501e+09,False,0,2023-02-04 16:51:29.527463,1.675501e+09,2023-02-04 16:51:29.593287,1.675501e+09
3,3509,2023-02-04 16:51:29.529455,1.675501e+09,False,0,2023-02-04 16:51:29.529462,1.675501e+09,2023-02-04 16:51:29.599267,1.675501e+09
4,3510,2023-02-04 16:51:29.531455,1.675501e+09,False,0,2023-02-04 16:51:29.531463,1.675501e+09,2023-02-04 16:51:29.599269,1.675501e+09
...,...,...,...,...,...,...,...,...,...
273154,276660,2023-02-04 17:00:35.856984,1.675501e+09,False,0,2023-02-04 17:00:35.856991,1.675501e+09,2023-02-04 17:00:35.932875,1.675501e+09
273155,276661,2023-02-04 17:00:35.858984,1.675501e+09,False,0,2023-02-04 17:00:35.858991,1.675501e+09,2023-02-04 17:00:35.932929,1.675501e+09
273156,276662,2023-02-04 17:00:35.860984,1.675501e+09,False,0,2023-02-04 17:00:35.860992,1.675501e+09,2023-02-04 17:00:35.932930,1.675501e+09
273157,276663,2023-02-04 17:00:35.862984,1.675501e+09,False,0,2023-02-04 17:00:35.862991,1.675501e+09,2023-02-04 17:00:35.932931,1.675501e+09


In [23]:
def get_latency_jitter(df, mode):
    # define latnecy := arrival.time - payload.time
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # payload.time
    df['transmit.time'] = pd.to_datetime(df['transmit.time'])
    df['arrival.time'] = pd.to_datetime(df['arrival.time'])

    ### calculate latency
    # print(df['latency'])
    # print(df['latency'][df['lost'] == False])
    df['latency'] = (df['arrival.time'] - df['Timestamp']).dt.total_seconds().round(6)
    # display(df)
    
    ### calculate latency
    # print(df['latency'])
    # print(df['latency'][df['lost'] == False])
    df['latency'] = (df['arrival.time'] - df['Timestamp']).dt.total_seconds().round(6)
    # display(df)
    
    latndf = df['latency'][df['lost'] == False]
    print("min latency:         ", min(latndf), "seconds")
    print("max latency:         ", max(latndf), "seconds")
    print("mean latency:        ", round(mean(latndf), 6), "seconds")
    print("stdev latency:       ", round(stdev(latndf), 6), "seconds")
    print("------------------------------------")
    
    bm = (5 + random.randint(-2000, 2000)*1e-3) * 1e-3
    latndf = df['latency'][df['lost'] == False]
    minlatn = min(latndf)
    epoch_comp = bm - minlatn
    comp = pd.to_timedelta(epoch_comp, "sec")
    
    print(epoch_comp)
    print(comp)
    print("------------------------------------")
    
    if mode == "dl":
        df['arrival.time_epoch'] = df['arrival.time_epoch'] + epoch_comp
        df['arrival.time'] = df['arrival.time'] + comp
    elif mode == "ul":
        df['Timestamp_epoch'] = df['Timestamp_epoch'] + epoch_comp
        df['Timestamp'] = df['Timestamp'] + comp
        df['transmit.time_epoch'] = df['transmit.time_epoch'] + epoch_comp
        df['transmit.time'] = df['transmit.time'] + comp
    
    df['latency'] = (df['arrival.time_epoch'] - df['Timestamp_epoch'])
    latndf = df['latency'][df['lost'] == False]
    print("min latency:         ", min(latndf), "seconds")
    print("max latency:         ", max(latndf), "seconds")
    print("mean latency:        ", round(mean(latndf), 6), "seconds")
    print("stdev latency:       ", round(stdev(latndf), 6), "seconds")
    
    # df['latency'][df['lost'] == False] = (df['arrival.time'][df['lost'] == False] - df['Timestamp'][df['lost'] == False]).dt.total_seconds().round(6)
    
    return df

In [24]:
df = get_latency_jitter(df, mode="dl")
# display(df)

/tmp/ipykernel_286755/964550962.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['lost'] == True]['latency'] = INF


min latency:          0.003198 seconds
max latency:          0.431801 seconds
mean latency:         0.010121 seconds
stdev latency:        0.013282 seconds
------------------------------------
0.0025100000000000005
0 days 00:00:00.002510
------------------------------------
min latency:          0.005708217620849609 seconds
max latency:          0.4343111515045166 seconds
mean latency:         0.012631 seconds
stdev latency:        0.013282 seconds


In [195]:
def get_statistics(df, fout1, fout2, fout3):
    # output packet record csv
    print("output >>>", fout1)
    # df.to_csv(fout1, index=False)
    
    # loss statistics
    total_packet_sent = len(df)
    total_loss = len(df[df["lost"] == True])
    loss_rate = total_loss / (total_packet_sent + 1e-9) * 100  # ratio (%)
    exp_time = round(df['Timestamp_epoch'].iloc[-1] - df['Timestamp_epoch'].iloc[0], 6) if total_packet_sent else 0
    print("output >>>", fout2)
    # with open(fout1, "w", newline='') as fp:
    #     writer = csv.writer(fp)
    #     writer.writerow(['total_packet_sent', 'total_packet_loss', 'packet_loss_rate(%)', 'experiment_time(sec)'])
    #     writer.writerow([total_packet_sent, total_loss, loss_rate, exp_time])
    
    # latency statistics
    latndf = df['latency'][df['lost'] == False]
    total_packet_recv = len(latndf)
    total_excs_latency = len(latndf[latndf > 100e-3])
    excs_latency_rate = total_excs_latency / (total_packet_recv + 1e-9) * 100  # ratio (%)
    
    print("output >>>", fout3)
    # with open(fout2, "w", newline='') as fp:
    #     writer = csv.writer(fp)
    #     writer.writerow(['total_packet_recv', 'total_excessive_latency', 'excessive_latency_rate(%)', 'experiment_time(sec)'])
    #     writer.writerow([total_packet_recv, total_excs_latency, excs_latency_rate, exp_time])
        
    # print(min(latndf), max(latndf), mean(latndf), stdev(latndf))
    
    print("------------------------------------------")
    print("min latency:         ", min(latndf), "seconds")
    print("max latency:         ", max(latndf), "seconds")
    print("mean latency:        ", round(mean(latndf), 6), "seconds")
    print("stdev latency:       ", round(stdev(latndf), 6), "seconds")
    print("total_packet_sent:   ", total_packet_sent)
    print("total_packet_recv:   ", total_packet_recv)
    print("total_packet_loss:   ", total_loss)
    print("packet_loss_rate(%): ", round(loss_rate, 3), "%")
    print("total_excs_latency:  ", total_excs_latency)
    print("excs_latency_rate(%):", round(excs_latency_rate, 3), "%")
    print("experiment_time(sec):", exp_time, "seconds")
    print("------------------------------------------")
    print()

In [196]:
get_statistics(df, fout1='', fout2='', fout3='')

output >>> 
output >>> 
output >>> 
------------------------------------------
min latency:          0.064392 seconds
max latency:          0.492995 seconds
mean latency:         0.071315 seconds
stdev latency:        0.013282 seconds
total_packet_sent:    273159
total_packet_recv:    272130
total_packet_loss:    1029
packet_loss_rate(%):  0.377 %
total_excs_latency:   3199
excs_latency_rate(%): 1.176 %
experiment_time(sec): 546.341529 seconds
------------------------------------------

